In [ ]:
%pip install requests

In [14]:
teams = ["TOR"] # Will include all teams later on

In [15]:
url_format = "https://www.basketball-reference.com/teams/{}/2025.html"

In [ ]:
%pip install beautifulsoup4

In [ ]:
!python -m pip install certifi

In [9]:
import certifi
import urllib3
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where()
)

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import os

# If you get an error, try printing response. If it returns <Reponse [429]>, you are sending too many requests, you have to wait a while to run again
# Check the reponse header to see how long you have to wait before running the req again (we can implement this automatically (optional)), usually 1 hour
# This function will take a while to prevent overflowing the limit (max 30 req /min)

# Since this takes so long, one team might need to be updated on seperate days / and maybe every week worst case

# We use certification to adhere with best practices (this doesn't work on corporate wifi)'
# Right now, it sets verification to false, uncomment and replace this line with the one below to use certification

# Initial runtime for the Raptors took 4 min 27.2 seconds 

for team in teams:
    url = url_format.format(team)
    time.sleep(2)
    response = requests.get(url, verify=False)
    #response = requests.get(url, verify=certifi.where())
    print(response)
    print(response.headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    roster =  soup.find("table", {"id": "roster"})

    for row in roster.find_all("tr"):
        player_cells = row.find_all('td', {'data-stat': 'player'})

        for cell in player_cells:
            link_tag = cell.find('a')
            if link_tag:
                player_link = link_tag['href']

                full_player_link_format = "https://www.basketball-reference.com{}"
                full_player_link = full_player_link_format.format(player_link)
                time.sleep(2)
                player_response = requests.get(full_player_link, verify=False)
                #player_response = requests.get(full_player_link, verify=certifi.where())
                player_soup = BeautifulSoup(player_response.content, 'html.parser')

                game_logs_header = player_soup.find('p', {'class': 'listhead'}, string="Game Logs")
                    
                if game_logs_header:
                    game_log_links = game_logs_header.find_next_sibling('ul')

                    if game_log_links:
                        for link in game_log_links.find_all('a', href=True):
                            nested_link = link['href']
                            print(f"Nested link: https://www.basketball-reference.com{nested_link}")

                            player_name = player_link.split('/')[-1].replace('.html', '')
                            game_log_link_format = "https://www.basketball-reference.com{}"
                            game_log_link = game_log_link_format.format(nested_link)
                            time.sleep(2)
                            game_log_response = requests.get(game_log_link, verify=False)

                            year = game_log_link.split('/')[-1]

                            if year == "":
                                year = "playoffs"

                            # Create the directory if it doesn't exist
                            directory = f"data/{team}/{player_name}"
                            if not os.path.exists(directory):
                                os.makedirs(directory)

                            # Replace this with a request to a relational database (MongoDB)
                            with open(f"{directory}/{year}.html", 'a', encoding='utf-8') as file:
                                file.write(game_log_response.text)

In [ ]:
%pip install pandas

In [ ]:
%pip install bs4
%pip install html5lib
%pip install lxml

In [18]:
import pandas as pd

In [ ]:
import os
import re
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup

# Does not work for playoff files yet (different table format)

dfs = []

outer_folder = 'data'

pattern = r"^(.*?)(?=\s\d{4}-\d{2}|\splayoffs)"

for dirpath, dirnames, filenames in os.walk(outer_folder):
    for filename in filenames:

        file_path = os.path.join(dirpath, filename)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            page = file.read()

        soup = BeautifulSoup(page, 'html.parser')

        # Remove middle headers
        middle_headers = soup.find_all('tr', class_="thead")

        for header in middle_headers:
            header.decompose()

        game_log_table = soup.find('table', {'id': 'pgl_basic'})

        print("File path: ", file_path)

        if game_log_table is None:
            print(f"No table with ID 'pgl_basic' found in file {file_path}")
            continue  # Skip this file and move to the next

        title_tag = soup.find('title')

        if title_tag:
            title_text = title_tag.get_text()
            match = re.search(pattern, title_text)

            if match:
                full_name = match.group(1).strip() 

        game_log = pd.read_html(StringIO(str(game_log_table)))[0]
       
        if full_name:
            game_log["Name"] = full_name

            # Reorder the columns to put "Name" first
            columns = ['Name'] + [col for col in game_log.columns if col != 'Name']
            game_log = game_log[columns]

        dfs.append(game_log)

In [42]:
game_logs = pd.concat(dfs)
game_logs.drop(['Unnamed: 5', "Rk", "G"], axis=1, inplace=True)  # Modifies df directly
game_logs.rename(columns={'Unnamed: 7': 'Win/Loss'}, inplace=True)


In [43]:
game_logs.to_csv("game_logs.csv")

In [ ]:
game_logs